# Zadanie 1

In [ ]:
"""Function definitions that are used in LSB steganography."""
from matplotlib import pyplot as plt
import numpy as np
import binascii
import cv2 as cv
import math
plt.rcParams["figure.figsize"] = (18,10)


def encode_as_binary_array(msg):
    """Encode a message as a binary string."""
    msg = msg.encode("utf-8")
    msg = msg.hex()
    msg = [msg[i:i + 2] for i in range(0, len(msg), 2)]
    msg = [ "{:08b}".format(int(el, base=16)) for el in msg]
    return "".join(msg)


def decode_from_binary_array(array):
    """Decode a binary string to utf8."""
    array = [array[i:i+8] for i in range(0, len(array), 8)]
    if len(array[-1]) != 8:
        array[-1] = array[-1] + "0" * (8 - len(array[-1]))
    array = [ "{:02x}".format(int(el, 2)) for el in array]
    array = "".join(array)
    result = binascii.unhexlify(array)
    return result.decode("utf-8", errors="replace")


def load_image(path, pad=False):
    """Load an image.
    
    If pad is set then pad an image to multiple of 8 pixels.
    """
    image = cv.imread(path)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    if pad:
        y_pad = 8 - (image.shape[0] % 8)
        x_pad = 8 - (image.shape[1] % 8)
        image = np.pad(
            image, ((0, y_pad), (0, x_pad) ,(0, 0)), mode='constant')
    return image


def save_image(path, image):
    """Save an image."""
    plt.imsave(path, image) 


def clamp(n, minn, maxn):
    """Clamp the n value to be in range (minn, maxn)."""
    return max(min(maxn, n), minn)


def hide_message(image, message, nbits=1):
    """Hide a message in an image (LSB).
    
    nbits: number of least significant bits
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()
    if len(message) > len(image) * nbits:
        raise ValueError("Message is to long :(")
    
    chunks = [message[i:i + nbits] for i in range(0, len(message), nbits)]
    for i, chunk in enumerate(chunks):
        byte = "{:08b}".format(image[i])
        new_byte = byte[:-nbits] + chunk
        image[i] = int(new_byte, 2)
        
    return image.reshape(shape)


def reveal_message(image, nbits=1, length=0):
    """Reveal the hidden message.
    
    nbits: number of least significant bits
    length: length of the message in bits.
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()
    length_in_pixels = math.ceil(length/nbits)
    if len(image) < length_in_pixels or length_in_pixels <= 0:
        length_in_pixels = len(image)
    
    message = ""
    i = 0
    while i < length_in_pixels:
        byte = "{:08b}".format(image[i])
        message += byte[-nbits:]
        i += 1
        
    mod = length % -nbits
    if mod != 0:
        message = message[:mod]
    return message


original_image = load_image("images/spanish.png")  # Wczytanie obrazka
# Mnożenie stringów działa jak zwielokratnianie
message = "Jak sie ciesze, ze w koncu dotarlem na zajecia z IOb!"
n = 1  # liczba najmłodszych bitów używanych do ukrycia wiadomości

message = encode_as_binary_array(message)  # Zakodowanie wiadomości jako ciąg 0 i 1
image_with_message = hide_message(original_image, message, n)  # Ukrycie wiadomości w obrazku

save_image("images/zad1.png", image_with_message)  # Zapisanie obrazka w formacie PNG

image_with_message_png = load_image("images/zad1.png")  # Wczytanie obrazka PNG

secret_message_png = decode_from_binary_array(
    reveal_message(image_with_message_png, nbits=n, length=len(message)))  # Odczytanie ukrytej wiadomości z PNG

print("Ukryta wiadomosc: " + secret_message_png)

# Wyświetlenie obrazków
f, ar = plt.subplots(1,3)
ar[0].imshow(original_image)
ar[0].set_title("Original image")
ar[0].axis('off')
ar[1].imshow(image_with_message)
ar[1].set_title("Image with message")
ar[1].axis('off')
ar[2].imshow(image_with_message_png)
ar[2].set_title("PNG image with message")
ar[2].axis('off')

# Zadanie 2

In [ ]:
import lorem

def calculate_mse(image1, image2):
    """Calculate Mean Square Error between two images."""
    image1 = np.asarray(image1, dtype=np.float32)
    image2 = np.asarray(image2, dtype=np.float32)
    mse = np.mean((image1 - image2) ** 2)
    return mse


original_image = load_image("images/spanish.png")
image_pixels = original_image.shape[0] * original_image.shape[1]
print(image_pixels)

message = lorem.text() * 15
message = encode_as_binary_array(message)  # Zakodowanie wiadomości jako ciąg 0 i 1
max_bits = 115000  # przyciecie message do odpowiedniej dlugosci, aby stanowilo ~80% obrazka
message = message[:max_bits]
message_bits = len(message)
print(message_bits)

print(str(message_bits/image_pixels))  # w celu sprawdzenia jaki procent obrazka stanowi ukrywana wiadomosc


mse_values = []

for i in range(1, 9):
    image_with_message = hide_message(original_image, message, i)
    mse = calculate_mse(original_image, image_with_message)
    mse_values.append(mse)

# Plotting the MSE diagram
plt.figure(figsize=(8, 5))
plt.plot(range(1, 9), mse_values, marker='o')
plt.xlabel("nbits")
plt.ylabel("Mean Squared Error (MSE)")
plt.title("MSE depending on the bits used for LSB Steganography")
plt.grid(True)
plt.tight_layout()
plt.show()


# Zadanie 3

In [ ]:
def hide_message(image, message, nbits=1, spos=0):
    """Hide a message in an image (LSB).
    
    nbits: number of least significant bits
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()

    if spos < 0 or spos >= len(image):
        raise ValueError("Invalid starting position (spos)")
    
    available_bits = (len(image) - spos) * nbits
    if len(message) > available_bits:
        raise ValueError("Message is too long to fit from spos")

    # Podzielenie wiadommosci na kawalki o rozmiarze nbits
    chunks = [message[i:i + nbits] for i in range(0, len(message), nbits)]
    
    for i, chunk in enumerate(chunks):
        idx = spos + i
        byte = "{:08b}".format(image[idx])  # zamiana bajtu obrazka na ciag 8 bitow
        new_byte = byte[:-nbits] + chunk.zfill(nbits)  # pad if needed
        image[idx] = int(new_byte, 2)  # 2 - string jest ciagiem binarnym

    return image.reshape(shape)

    
def reveal_message(image, nbits=1, length=0, spos=0):
    """Reveal the hidden message.
    
    nbits: number of least significant bits
    length: length of the message in bits.
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()

    if spos < 0 or spos >= len(image):
        raise ValueError("Invalid starting position (spos)")

    available_pixels = len(image) - spos
    length_in_pixels = math.ceil(length / nbits) if length > 0 else available_pixels
    
    if length_in_pixels > available_pixels:
        length_in_pixels = available_pixels
    
    message = ""
    i = 0
    for i in range(length_in_pixels):
        byte = "{:08b}".format(image[spos + i])
        message += byte[-nbits:]
        
    if length > 0 and len(message) > length:
        message = message[:length]

    return message


original_image = load_image("images/spanish.png")  # Wczytanie obrazka
message = "Jak sie ciesze, ze w koncu dotarlem na zajecia z IOb!"
n = 1  # liczba najmłodszych bitów używanych do ukrycia wiadomości
start_pos = 30  # pozycja poczatkowa ukrywanej wiadomosci

message = encode_as_binary_array(message)  # Zakodowanie wiadomości jako ciąg 0 i 1
image_with_message = hide_message(original_image, message, n, start_pos)  # Ukrycie wiadomości w obrazku

save_image("images/zad3.png", image_with_message)  # Zapisanie obrazka w formacie PNG

image_with_message_png = load_image("images/zad3.png")  # Wczytanie obrazka PNG

secret_message_png = decode_from_binary_array(
    reveal_message(image_with_message_png, nbits=n, length=len(message), spos=start_pos))  # Odczytanie ukrytej wiadomości z PNG

print("Ukryta wiadomosc: " + secret_message_png)

f, ar = plt.subplots(1,3)
ar[0].imshow(original_image)
ar[0].set_title("Original image")
ar[1].imshow(image_with_message)
ar[1].set_title("Image with message")
ar[2].imshow(image_with_message_png)
ar[2].set_title("PNG image with message")

# Zadanie 4

In [ ]:
from PIL import Image
import io

def hide_image(image, secret_image_path, nbits=1):
    with open(secret_image_path, "rb") as file:
        secret_img = file.read()
        
    secret_img = secret_img.hex()
    secret_img = [secret_img[i:i + 2] for i in range(0, len(secret_img), 2)]
    secret_img = ["{:08b}".format(int(el, base=16)) for el in secret_img]
    secret_img = "".join(secret_img)
    return hide_message(image, secret_img, nbits), len(secret_img)


def retrieve_image(image, length, nbits=1):
    """Retrieve a hidden image."""
    
    # Wyciagniecie ukrytego ciagu binarnego z obrazka
    binary_image = reveal_message(image, nbits=nbits, length=length)
    
    # Konwersja ciagu binarnego na tablice bajtow
    bytes_image = [binary_image[i:i + 8] for i in range(0, len(binary_image), 8)]
    bytes_image = bytes([int(b, 2) for b in bytes_image])

    # Wczytanie obrazka z tablicy bajtow
    try:
        hidden_image = Image.open(io.BytesIO(bytes_image))
        return hidden_image
    except Exception as e:
        raise ValueError("Failed to decode the hidden image: " + str(e))


image = load_image("images/rembrandt.png")
image_with_secret, length_of_secret = hide_image(image, "images/spanish.jpg", 1)

hidden_image = retrieve_image(image_with_secret, length_of_secret, 1)

f, ar = plt.subplots(1,2)
ar[0].imshow(image_with_secret)
ar[0].set_title("Image with secret hiden")
ar[0].axis('off')
ar[1].imshow(hidden_image)
ar[1].set_title("Retrieved secret image")
ar[1].axis('off')

# Zadanie 5

In [ ]:
def retrieve_image(image, nbits=1):
    """Retrieve a hidden image, without needing its length."""
    
    # Wyciagniecie wszystkich mozliwych LSB
    binary_data = reveal_message(image, nbits=nbits, length=0)
    
    # Konwersja ciagu binarnego na tablice bajtow
    byte_data = [binary_data[i:i + 8] for i in range(0, len(binary_data), 8)]
    try:
        byte_array = bytes([int(b, 2) for b in byte_data])
    except ValueError:
        raise ValueError("Binary data could not be converted to bytes.")
    
    # Szukamy znacznika JPEG EOF (FFD9)
    eof_marker = b'\xFF\xD9'
    eof_index = byte_array.find(eof_marker)
    if eof_index == -1:
        raise ValueError("JPEG EOF marker not found.")

    # Przyciecie zawartosci tablicy bajtow po wystapieniu znacznika EOF
    hidden_bytes = byte_array[:eof_index + 2]

    # Wczytanie obrazka z tablicy bajtow
    try:
        hidden_img = Image.open(io.BytesIO(hidden_bytes))
        return hidden_img
    except Exception as e:
        raise ValueError("Failed to decode the hidden image: " + str(e))


image = load_image("images/rembrandt.png")
image_with_secret, _ = hide_image(image, "images/spanish.jpg", 1)

hidden_image = retrieve_image(image_with_secret, nbits=1)

f, ar = plt.subplots(1,2)
ar[0].imshow(image_with_secret)
ar[0].set_title("Image with secret hidden")
ar[1].imshow(hidden_image)
ar[1].set_title("Retrieved secret image")